In [ ]:
# APIKEY = "secret"
# NAME = "Dublin"
# STATIONS_URI = "https://api.jcdecaux.com/vls/v1/stations/?apiKey=603f7ff1122cbceaaca8a9f164e69f9d48b19a51&contract=Dublin"

# r = requests.get(STATIONS_URI, params={"apiKey": '603f7ff1122cbceaaca8a9f164e69f9d48b19a51',"contract": NAME})

In [ ]:
import requests

In [ ]:
result = requests.get('https://api.jcdecaux.com/vls/v1/stations/?apiKey=603f7ff1122cbceaaca8a9f164e69f9d48b19a51&contract=Dublin')

In [ ]:
result.status_code

In [ ]:
station= result.json()

In [24]:
result.json()

[{'number': 42,
  'contract_name': 'dublin',
  'name': 'SMITHFIELD NORTH',
  'address': 'Smithfield North',
  'position': {'lat': 53.349562, 'lng': -6.278198},
  'banking': True,
  'bonus': False,
  'bike_stands': 30,
  'available_bike_stands': 10,
  'available_bikes': 20,
  'status': 'OPEN',
  'last_update': 1614073837000},
 {'number': 30,
  'contract_name': 'dublin',
  'name': 'PARNELL SQUARE NORTH',
  'address': 'Parnell Square North',
  'position': {'lat': 53.353462, 'lng': -6.265305},
  'banking': True,
  'bonus': False,
  'bike_stands': 20,
  'available_bike_stands': 15,
  'available_bikes': 5,
  'status': 'OPEN',
  'last_update': 1614074090000},
 {'number': 54,
  'contract_name': 'dublin',
  'name': 'CLONMEL STREET',
  'address': 'Clonmel Street',
  'position': {'lat': 53.336021, 'lng': -6.26298},
  'banking': False,
  'bonus': False,
  'bike_stands': 33,
  'available_bike_stands': 25,
  'available_bikes': 8,
  'status': 'OPEN',
  'last_update': 1614074034000},
 {'number': 108,


In [ ]:
import pymysql
db = pymysql.connect(host='dublinbikes.ctjs1bdwfyjm.us-east-1.rds.amazonaws.com', port=int(3306), user="rohit", passwd="rohit1996")
cursor=db.cursor()

In [ ]:
import mysql.connector

#establishing the connection
conn = mysql.connector.connect(
   user='rohit', password='rohit1996', host='dublinbikes.ctjs1bdwfyjm.us-east-1.rds.amazonaws.com', database='dbikes'
)

#Creating a cursor object using the cursor() method
cursor = conn.cursor()

#Dropping station table if already exists.
cursor.execute("DROP TABLE IF EXISTS station")

#Creating table as per requirement
sql_station = """ 
CREATE TABLE IF NOT EXISTS station (
number INTEGER,
address VARCHAR (256),
banking INTEGER,
bike_stands INTEGER,
position_lat REAL,
position_lng REAL,
bonus INTEGER,
contract_name VARCHAR(256),
name VARCHAR(256),
status VARCHAR(256)
)

"""
cursor.execute(sql_station)
conn.commit()

In [34]:
cursor.execute("DROP TABLE IF EXISTS availability")
sql_availability = """ 
CREATE TABLE IF NOT EXISTS availability (
id INT AUTO_INCREMENT PRIMARY KEY,
number INTEGER,
available_bikes_stands INTEGER,
available_bikes INTEGER,
last_update INTEGER
)
"""

cursor.execute(sql_availability)
conn.commit()

In [32]:
def get_station (obj):
       return {'number':obj['number'],
           'address':obj['address'],
            'banking':obj['banking'],
            'bike_stands':obj['bike_stands'],
            'position_lat':obj['position']['lat'],
            'position_lng':obj['position']['lng'],
            'bonus':obj['bonus'],
            'contract_name':obj['contract_name'],
            'name':obj['name'],
            'status':obj['status']
           }
values = list(map(get_station,result.json()))

for station in values:

        vals = (station.get('number'), station.get('address'), int(station.get('banking')), station.get('bike_stands'),
               station.get('position_lat'), station.get('position_lng'), int(station.get('bonus')), station.get('contract_name'), 
               station.get('name'), station.get('status'))
        cursor.execute("insert into dbikes.station(number, address, banking, bike_stands,position_lat, position_lng, bonus, contract_name, name, status) values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)", vals)
        conn.commit()


In [35]:
def get_availablity (obj):
       return {'number':obj['number'],
           'available_bikes_stands':obj['available_bikes_stands'],
            'available_bikes':obj['available_bikes'],
            'last_update':obj['last_update']
           }
values = list(map(get_availablity,result.json()))
# print(values)
for availablity in values:
        print(availablity)
        vals = (availablity.get('number'), availablity.get('available_bikes_stands'), availablity.get('available_bikes'),
               availablity.get('last_update'))
        cursor.execute("insert into dbikes.availablity(number, available_bikes_stands, available_bikes, last_update) values(%s,%s,%s,%s)", vals)
        conn.commit()

KeyError: 'available_bikes_stands'